### Imports

In [75]:
import os
import pandas as pd
import config
import json
from fetch_messages import retrieve_messages
from IPython.display import display

### Scrape messages
- Set the `channel_id` you want to scrape from in your `config.py`
- `to_fetch` defaults to -1, which tells the scraper to grab as much as it can

In [ ]:
retrieve_messages(channel_id=config.general_channel_id, to_fetch=-1)

### Load messages into dataframe
- Also writes the messages to a .csv

In [53]:
dfs = []
for filename in os.listdir('messages'):
    data = pd.read_json(f'messages/{filename}')
    dfs.append(data)

messages_df = pd.concat(dfs, ignore_index=True)

In [ ]:
display(messages_df)
messages_df.to_csv(f'messages/messages.csv')

Reload dataframe from .csv

In [57]:
messages_df = pd.read_csv('messages/messages.csv')

### Remove embedded messages

In [130]:
messages_df = messages_df[messages_df['attachments']=='[]']
messages_df = messages_df[messages_df['embeds']=='[]']
messages_df = messages_df[messages_df['application_id'].isnull()]

### Profile users
- Creates a `{'username': 'id'}` dictionary containing each user that has sent a message

In [127]:
user_temp_df = messages_df.copy()
user_temp_df['author'] = user_temp_df['author'].apply(eval)

users_dict = {}
while not user_temp_df.empty:
    author = user_temp_df.iloc[0]['author']
    users_dict[author['username']] = author['id']
    user_temp_df = user_temp_df[user_temp_df['author'].apply(lambda x: x['id']) != author['id']]

In [ ]:
print(users_dict)